# 新規材料の予測

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid', font_scale=1.2)  # seabornスタイル適用
import numpy as np
import pandas as pd

from sklearn.cross_decomposition import PLSRegression

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, KFold, cross_val_predict
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
import lightgbm as lgb

import optuna

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import pickle

from scipy.spatial.distance import cdist
from sklearn.svm import OneClassSVM
from sklearn.decomposition import PCA

/opt/conda/envs/mol-regression-env/lib/python3.9/site-packages/xgboost/compat.py:105: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


## データ読み込み＆前処理

In [2]:
# データ読み込み、学習データ＋未知のデータ
data = pd.read_csv('material_data.csv', index_col=0)
# rdkit
des_rdkit = pd.read_csv('descriptor_rdkit.csv', index_col=0)
# FP
fingerprint_df = pd.read_csv('morganFP.csv', index_col=0)
# mordred, 2次元
des_mordred_2d = pd.read_csv('descriptor_mordred_2d.csv', index_col=0)
# mordred, 3次元
des_mordred_3d = pd.read_csv('descriptor_mordred_3d.csv', index_col=0)

data.shape, des_rdkit.shape, fingerprint_df.shape, des_mordred_2d.shape, des_mordred_3d.shape

((233, 3), (233, 217), (233, 2048), (233, 1158), (233, 1826))

In [3]:
# 結合
dataset = pd.concat([data.reset_index(), des_rdkit.reset_index(drop=True)], axis=1)

dataset.index = dataset['Material']
dataset = dataset.drop(dataset.columns[0], axis=1)

# TypeとSMILESも消す
dataset = dataset.drop(['SMILES', 'Type'], axis=1)

# 学習用と予測用に分ける
dataset_train = dataset.dropna(subset='PL')
dataset_test = dataset[dataset['PL'].isnull()]

# 予測データのPLは空なので消す
dataset_test = dataset_test.drop('PL', axis=1)

# infをNaNに置き換え
dataset_train = dataset_train.replace(np.inf, np.nan).fillna(np.nan)
dataset_train = dataset_train.drop(dataset_train.columns[dataset_train.isnull().any()], axis=1)

dataset_test = dataset_test.replace(np.inf, np.nan).fillna(np.nan)
dataset_test = dataset_test.drop(dataset_test.columns[dataset_test.isnull().any()], axis=1)

# 学習データのstdが0の列を特定
zero_std_cols = dataset_train.columns[dataset_train.std() == 0]

# 学習・未知データから同じ列を削除
dataset_train = dataset_train.drop(columns=zero_std_cols)
dataset_test = dataset_test.drop(columns=zero_std_cols, errors='ignore')  # 念のため

print(dataset_train.shape, dataset_test.shape)

(228, 152) (5, 151)


In [4]:
# 最終的な特徴量共通セット
X_train_all = dataset_train.drop(columns='PL')
y_train_all = dataset_train['PL']
X_test = dataset_test  # PLなし

## NN

In [5]:
# GPU set
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [12]:
# データセット定義
class RegressionDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).view(-1, 1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
# 学習loop
def train(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for X_batch, y_batch in dataloader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        pred = model(X_batch)
        loss = criterion(pred, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(dataloader)

# 評価
def evaluate(model, dataloader, device, y_scaler):
    model.eval()
    preds = []
    trues = []
    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.cpu().numpy()
            pred = model(X_batch).cpu().numpy()
            preds.append(pred)
            trues.append(y_batch)

    y_pred_std = np.vstack(preds)
    y_true_std = np.vstack(trues)

    # 逆標準化
    y_pred = y_scaler.inverse_transform(y_pred_std)
    y_true = y_scaler.inverse_transform(y_true_std)

    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return rmse

In [13]:
# Optuna設定
def define_model(trial, input_dim):
    n_layers = trial.suggest_int("n_layers", 1, 5)
    hidden_dim = trial.suggest_int("hidden_dim", 64, 512)
    dropout_rate = trial.suggest_float("dropout_rate", 0.0, 0.5)
    activation = trial.suggest_categorical("activation", ["relu", "leaky_relu"])

    layers = []
    for i in range(n_layers):
        layers.append(nn.Linear(input_dim if i == 0 else hidden_dim, hidden_dim))
        layers.append(nn.ReLU() if activation == "relu" else nn.LeakyReLU())
        layers.append(nn.Dropout(dropout_rate))
    layers.append(nn.Linear(hidden_dim, 1))
    return nn.Sequential(*layers)



def objective(trial):
    # ハイパーパラメータ提案
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD"])

    kf = KFold(n_splits=5, shuffle=True, random_state=1234)
    val_losses = []

    # 分割前に標準化してしまう
    X_scaler = StandardScaler()
    autoscaled_X = X_scaler.fit_transform(X_train_all)
    y_scaler = StandardScaler()
    autoscaled_y = y_scaler.fit_transform(y_train_all.values.reshape(-1, 1))

    for train_idx, val_idx in kf.split(autoscaled_X):
        # スライス
        X_train, X_val = autoscaled_X[train_idx], autoscaled_X[val_idx]
        y_train, y_val = autoscaled_y[train_idx], autoscaled_y[val_idx]

        # Dataset, DataLoader
        train_ds = RegressionDataset(X_train, y_train)
        val_ds = RegressionDataset(X_val, y_val)

        train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
        val_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

        # モデル定義
        model = define_model(trial, X_train.shape[1]).to(device)

        criterion = nn.MSELoss()

        optimizer = torch.optim.Adam(model.parameters(), lr=lr) if optimizer_name == "Adam" else torch.optim.SGD(model.parameters(), lr=lr)

        # 学習ループ
        for epoch in range(50):
            train(model, train_dl, optimizer, criterion, device)

        # 評価
        val_loss = evaluate(model, val_dl, device, y_scaler)
        val_losses.append(val_loss)

    return np.mean(val_losses)


In [14]:
# 実行
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

print('Best trial :', study.best_trial.params)

[I 2025-07-03 20:37:24,114] A new study created in memory with name: no-name-a9857564-e4aa-4cfa-b994-381d2fdc60fe
[I 2025-07-03 20:37:35,952] Trial 0 finished with value: 80.50936827247504 and parameters: {'batch_size': 16, 'lr': 0.00926298504585722, 'optimizer': 'Adam', 'n_layers': 3, 'hidden_dim': 386, 'dropout_rate': 0.2938193630456313, 'activation': 'leaky_relu'}. Best is trial 0 with value: 80.50936827247504.
[I 2025-07-03 20:37:43,957] Trial 1 finished with value: 98.63079913644268 and parameters: {'batch_size': 32, 'lr': 0.0029308610582944594, 'optimizer': 'SGD', 'n_layers': 5, 'hidden_dim': 416, 'dropout_rate': 0.40553641043943056, 'activation': 'relu'}. Best is trial 0 with value: 80.50936827247504.
[I 2025-07-03 20:37:48,814] Trial 2 finished with value: 79.53407312622943 and parameters: {'batch_size': 32, 'lr': 0.0016547827932387412, 'optimizer': 'SGD', 'n_layers': 3, 'hidden_dim': 213, 'dropout_rate': 0.10483916209291633, 'activation': 'leaky_relu'}. Best is trial 2 with va

Best trial : {'batch_size': 16, 'lr': 0.0011851505738656768, 'optimizer': 'Adam', 'n_layers': 5, 'hidden_dim': 437, 'dropout_rate': 0.10246354938881719, 'activation': 'relu'}


In [15]:
# 汎化性能チェック
# 評価指標格納用
rmse_scores = []
mae_scores = []
r2_scores = []

# 分割前に標準化してしまう
X_scaler = StandardScaler()
autoscaled_X = X_scaler.fit_transform(X_train_all)
y_scaler = StandardScaler()
autoscaled_y = y_scaler.fit_transform(y_train_all.values.reshape(-1, 1))

# 5分割交差検証
kf = KFold(n_splits=5, shuffle=True, random_state=1234)

for train_index, val_index in kf.split(autoscaled_X):
    # データ分割
    X_train, X_val = autoscaled_X[train_index], autoscaled_X[val_index]
    y_train, y_val = autoscaled_y[train_index], autoscaled_y[val_index]

    # Dataset/DataLoader
    train_ds = RegressionDataset(X_train, y_train)
    val_ds = RegressionDataset(X_val, y_val)
    train_dl = DataLoader(train_ds, batch_size=study.best_trial.params['batch_size'], shuffle=True)
    val_dl = DataLoader(val_ds, batch_size=study.best_trial.params['batch_size'], shuffle=False)

    # モデルと最適化
    model_nn_op = define_model(study.best_trial, input_dim=X_train.shape[1]).to(device)
    if study.best_trial.params['optimizer'] == "Adam":
        optimizer = torch.optim.Adam(model_nn_op.parameters(), lr=study.best_trial.params['lr'])
    else:
        optimizer = torch.optim.SGD(model_nn_op.parameters(), lr=study.best_trial.params['lr'])
    criterion = torch.nn.MSELoss()

    # 学習
    for epoch in range(30):  # 固定でOKなら30
        model_nn_op.train()
        for X_batch, y_batch in train_dl:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            pred = model_nn_op(X_batch)
            loss = criterion(pred, y_batch)
            loss.backward()
            optimizer.step()

    # 標準化された予測 → 逆変換
    model_nn_op.eval()
    preds = []
    trues = []

    with torch.no_grad():
        for X_batch, y_batch in val_dl:
            X_batch = X_batch.to(device)
            y_batch = y_batch.cpu().numpy()
            y_pred_std = model_nn_op(X_batch).cpu().numpy()
            preds.append(y_pred_std)
            trues.append(y_batch)

    y_pred_std = np.vstack(preds)
    y_true_std = np.vstack(trues)

    # 逆標準化
    y_pred = y_scaler.inverse_transform(y_pred_std)
    y_true = y_scaler.inverse_transform(y_true_std)

    # 評価（同じスケールで比較）
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    print(f'Fold RMSE : {rmse:.4f}')
    print(f'Fold MAE : {mae:.4f}')
    print(f'Fold R2  : {r2:.4f}\n')

    # 結果格納
    rmse_scores.append(rmse)
    mae_scores.append(mae)
    r2_scores.append(r2)

# 平均結果出力
print(f'\n平均RMSE : {np.mean(rmse_scores):.4f}')
print(f'平均MAE  : {np.mean(mae_scores):.4f}')
print(f'平均R2   : {np.mean(r2_scores):.4f}')


Fold RMSE : 42.3110
Fold MAE : 30.3051
Fold R2  : 0.8063

Fold RMSE : 56.6759
Fold MAE : 40.0587
Fold R2  : 0.7389

Fold RMSE : 45.6701
Fold MAE : 32.2292
Fold R2  : 0.7978

Fold RMSE : 34.1775
Fold MAE : 25.9352
Fold R2  : 0.8527

Fold RMSE : 38.6853
Fold MAE : 28.7894
Fold R2  : 0.8158


平均RMSE : 43.5039
平均MAE  : 31.4635
平均R2   : 0.8023
